In [1]:
from gensim.models import word2vec
from opencc import OpenCC
import jieba
import re
import logging
import json
import multiprocessing

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
def read():
    cc = OpenCC('s2twp')
    
    files = ['AA','AB','AC','AD','AE','AF','AG','AH','AI','AJ','AK','AL']
    text = []
    for fi in files:
        for i in range(0, 100):
            if i%10==0:
                print('./wiki_zh/{}/wiki_{}'.format(fi,str(i).zfill(2)))
            with open('./wiki_zh/{}/wiki_{}'.format(fi,str(i).zfill(2)),'r', encoding = 'utf-8') as file_open:
                data = []
                for line in file_open:
                    data = json.loads(line)
                    x = re.sub('\s+','',cc.convert(data['text']))
                    text.append(re.sub('[^\u4e00-\u9fa5]+',' ', x) + '\n')
    for j in range(0,74):
        with open('./wiki_zh/AM/wiki_{}'.format(str(j).zfill(2)),'r', encoding = 'utf-8') as file_open:
            if j%10==0:
                print('./wiki_zh/AM/wiki_{}'.format(str(j).zfill(2)))
            data = []
            for line in file_open:
                data = json.loads(line)
                x = re.sub('\s+','',cc.convert(data['text']))
                text.append(x + '\n')
    
    with open('output.txt', 'w+') as f:
        for i in text:
            f.write(i)
            
read()

./wiki_zh/AA/wiki_00
./wiki_zh/AA/wiki_10
./wiki_zh/AA/wiki_20
./wiki_zh/AA/wiki_30
./wiki_zh/AA/wiki_40
./wiki_zh/AA/wiki_50
./wiki_zh/AA/wiki_60
./wiki_zh/AA/wiki_70
./wiki_zh/AA/wiki_80
./wiki_zh/AA/wiki_90
./wiki_zh/AB/wiki_00
./wiki_zh/AB/wiki_10
./wiki_zh/AB/wiki_20
./wiki_zh/AB/wiki_30
./wiki_zh/AB/wiki_40
./wiki_zh/AB/wiki_50
./wiki_zh/AB/wiki_60
./wiki_zh/AB/wiki_70
./wiki_zh/AB/wiki_80
./wiki_zh/AB/wiki_90
./wiki_zh/AC/wiki_00
./wiki_zh/AC/wiki_10
./wiki_zh/AC/wiki_20
./wiki_zh/AC/wiki_30
./wiki_zh/AC/wiki_40
./wiki_zh/AC/wiki_50
./wiki_zh/AC/wiki_60
./wiki_zh/AC/wiki_70
./wiki_zh/AC/wiki_80
./wiki_zh/AC/wiki_90
./wiki_zh/AD/wiki_00
./wiki_zh/AD/wiki_10
./wiki_zh/AD/wiki_20
./wiki_zh/AD/wiki_30
./wiki_zh/AD/wiki_40
./wiki_zh/AD/wiki_50
./wiki_zh/AD/wiki_60
./wiki_zh/AD/wiki_70
./wiki_zh/AD/wiki_80
./wiki_zh/AD/wiki_90
./wiki_zh/AE/wiki_00
./wiki_zh/AE/wiki_10
./wiki_zh/AE/wiki_20
./wiki_zh/AE/wiki_30
./wiki_zh/AE/wiki_40
./wiki_zh/AE/wiki_50
./wiki_zh/AE/wiki_60
./wiki_zh/AE/

In [4]:
def cutWords():
    logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.INFO)
    jieba.set_dictionary('./dict.txt')
    jieba.load_userdict('./user_dict.txt')
    stopword_set = set()
    with open('./stop_words.txt','r', encoding='utf-8') as stopwords:
        for stopword in stopwords:
            stopword_set.add(stopword.strip('\n'))
    
    output = open('cutDone.txt', 'w', encoding='utf-8')
    with open('output.txt', 'r', encoding='utf-8') as content :
        for texts_num, line in enumerate(content):
            line = line.strip('\n')
            words = jieba.cut(line, cut_all=False)
            for word in words:
                if word not in stopword_set:
                    output.write(word + ' ')
            output.write('\n')

            if (texts_num + 1) % 100000 == 0:
                logging.info("完成 %d 行斷詞" % (texts_num + 1))

cutWords()

Building prefix dict from /Users/kaihanandy/Desktop/dict.txt ...
2021-08-19 11:45:56,309 : Building prefix dict from /Users/kaihanandy/Desktop/dict.txt ...
Loading model from cache /var/folders/3c/l9sqbbm95k9d1nc91kjwx9wh0000gp/T/jieba.uf751ac817067c667147781ce70477c70.cache
2021-08-19 11:45:56,312 : Loading model from cache /var/folders/3c/l9sqbbm95k9d1nc91kjwx9wh0000gp/T/jieba.uf751ac817067c667147781ce70477c70.cache
Loading model cost 0.658 seconds.
2021-08-19 11:45:56,970 : Loading model cost 0.658 seconds.
Prefix dict has been built successfully.
2021-08-19 11:45:56,971 : Prefix dict has been built successfully.
2021-08-19 11:54:07,033 : 完成 100000 行斷詞
2021-08-19 11:58:37,665 : 完成 200000 行斷詞
2021-08-19 12:01:46,926 : 完成 300000 行斷詞
2021-08-19 12:04:54,999 : 完成 400000 行斷詞
2021-08-19 12:06:13,060 : 完成 500000 行斷詞
2021-08-19 12:07:26,137 : 完成 600000 行斷詞
2021-08-19 12:09:02,227 : 完成 700000 行斷詞
2021-08-19 12:12:09,015 : 完成 800000 行斷詞
2021-08-19 12:18:00,134 : 完成 900000 行斷詞
2021-08-19 12:20

In [5]:
def modeling():

    logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.INFO)
    sentences = word2vec.LineSentence(u"cutDone.txt")
    model = word2vec.Word2Vec(sentences,vector_size=400,epochs=5,workers=multiprocessing.cpu_count(),sg=1)
    
    model.save("ENtest.model")

modeling()

2021-08-19 12:28:07,523 : collecting all words and their counts
2021-08-19 12:28:07,528 : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-08-19 12:28:10,311 : PROGRESS: at sentence #10000, processed 8270256 words, keeping 326261 word types
2021-08-19 12:28:12,584 : PROGRESS: at sentence #20000, processed 14808480 words, keeping 471076 word types
2021-08-19 12:28:14,954 : PROGRESS: at sentence #30000, processed 20150284 words, keeping 583828 word types
2021-08-19 12:28:16,726 : PROGRESS: at sentence #40000, processed 24918333 words, keeping 668566 word types
2021-08-19 12:28:17,945 : PROGRESS: at sentence #50000, processed 28948375 words, keeping 734736 word types
2021-08-19 12:28:19,646 : PROGRESS: at sentence #60000, processed 32828041 words, keeping 799541 word types
2021-08-19 12:28:22,419 : PROGRESS: at sentence #70000, processed 36381386 words, keeping 856451 word types
2021-08-19 12:28:24,895 : PROGRESS: at sentence #80000, processed 38795526 words, keeping

2021-08-19 12:28:57,627 : PROGRESS: at sentence #740000, processed 127274188 words, keeping 2038031 word types
2021-08-19 12:28:58,035 : PROGRESS: at sentence #750000, processed 128599061 words, keeping 2049101 word types
2021-08-19 12:28:58,477 : PROGRESS: at sentence #760000, processed 130031804 words, keeping 2061231 word types
2021-08-19 12:28:59,039 : PROGRESS: at sentence #770000, processed 131871246 words, keeping 2077992 word types
2021-08-19 12:28:59,477 : PROGRESS: at sentence #780000, processed 133315812 words, keeping 2094414 word types
2021-08-19 12:28:59,887 : PROGRESS: at sentence #790000, processed 134648058 words, keeping 2107153 word types
2021-08-19 12:29:00,396 : PROGRESS: at sentence #800000, processed 136248565 words, keeping 2122029 word types
2021-08-19 12:29:00,940 : PROGRESS: at sentence #810000, processed 137642569 words, keeping 2134969 word types
2021-08-19 12:29:01,352 : PROGRESS: at sentence #820000, processed 138937335 words, keeping 2146653 word types
2

2021-08-19 12:29:51,071 : EPOCH 1 - PROGRESS: at 0.29% examples, 150246 words/s, in_qsize 7, out_qsize 0
2021-08-19 12:29:52,084 : EPOCH 1 - PROGRESS: at 0.31% examples, 151293 words/s, in_qsize 7, out_qsize 0
2021-08-19 12:29:53,093 : EPOCH 1 - PROGRESS: at 0.33% examples, 151233 words/s, in_qsize 7, out_qsize 0
2021-08-19 12:29:54,130 : EPOCH 1 - PROGRESS: at 0.35% examples, 151698 words/s, in_qsize 7, out_qsize 0
2021-08-19 12:29:55,170 : EPOCH 1 - PROGRESS: at 0.43% examples, 152316 words/s, in_qsize 7, out_qsize 0
2021-08-19 12:29:56,217 : EPOCH 1 - PROGRESS: at 0.56% examples, 152784 words/s, in_qsize 7, out_qsize 0
2021-08-19 12:29:57,220 : EPOCH 1 - PROGRESS: at 0.58% examples, 153072 words/s, in_qsize 7, out_qsize 0
2021-08-19 12:29:58,265 : EPOCH 1 - PROGRESS: at 0.59% examples, 152888 words/s, in_qsize 7, out_qsize 0
2021-08-19 12:29:59,271 : EPOCH 1 - PROGRESS: at 0.62% examples, 153300 words/s, in_qsize 7, out_qsize 0
2021-08-19 12:30:00,306 : EPOCH 1 - PROGRESS: at 0.64% 

2021-08-19 12:31:13,345 : EPOCH 1 - PROGRESS: at 2.57% examples, 160573 words/s, in_qsize 7, out_qsize 0
2021-08-19 12:31:14,359 : EPOCH 1 - PROGRESS: at 2.60% examples, 160708 words/s, in_qsize 7, out_qsize 0
2021-08-19 12:31:15,369 : EPOCH 1 - PROGRESS: at 2.63% examples, 160792 words/s, in_qsize 8, out_qsize 0
2021-08-19 12:31:16,444 : EPOCH 1 - PROGRESS: at 2.66% examples, 160859 words/s, in_qsize 7, out_qsize 0
2021-08-19 12:31:17,518 : EPOCH 1 - PROGRESS: at 2.69% examples, 160756 words/s, in_qsize 7, out_qsize 1
2021-08-19 12:31:18,657 : EPOCH 1 - PROGRESS: at 2.73% examples, 160881 words/s, in_qsize 7, out_qsize 0
2021-08-19 12:31:19,678 : EPOCH 1 - PROGRESS: at 2.76% examples, 161033 words/s, in_qsize 7, out_qsize 0
2021-08-19 12:31:20,682 : EPOCH 1 - PROGRESS: at 2.79% examples, 161171 words/s, in_qsize 7, out_qsize 0
2021-08-19 12:31:21,720 : EPOCH 1 - PROGRESS: at 2.82% examples, 161124 words/s, in_qsize 7, out_qsize 0
2021-08-19 12:31:22,801 : EPOCH 1 - PROGRESS: at 2.86% 

2021-08-19 12:32:35,544 : EPOCH 1 - PROGRESS: at 5.78% examples, 163449 words/s, in_qsize 7, out_qsize 0
2021-08-19 12:32:36,572 : EPOCH 1 - PROGRESS: at 5.82% examples, 163529 words/s, in_qsize 7, out_qsize 0
2021-08-19 12:32:37,675 : EPOCH 1 - PROGRESS: at 5.86% examples, 163523 words/s, in_qsize 7, out_qsize 0
2021-08-19 12:32:38,720 : EPOCH 1 - PROGRESS: at 5.91% examples, 163538 words/s, in_qsize 7, out_qsize 0
2021-08-19 12:32:39,737 : EPOCH 1 - PROGRESS: at 5.96% examples, 163567 words/s, in_qsize 7, out_qsize 0
2021-08-19 12:32:40,751 : EPOCH 1 - PROGRESS: at 6.01% examples, 163672 words/s, in_qsize 7, out_qsize 0
2021-08-19 12:32:41,787 : EPOCH 1 - PROGRESS: at 6.05% examples, 163661 words/s, in_qsize 7, out_qsize 0
2021-08-19 12:32:42,807 : EPOCH 1 - PROGRESS: at 6.11% examples, 163707 words/s, in_qsize 7, out_qsize 0
2021-08-19 12:32:43,888 : EPOCH 1 - PROGRESS: at 6.16% examples, 163761 words/s, in_qsize 7, out_qsize 0
2021-08-19 12:32:44,934 : EPOCH 1 - PROGRESS: at 6.21% 

...

2021-08-19 13:53:15,181 : EPOCH 5 - PROGRESS: at 56.19% examples, 155251 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:53:16,249 : EPOCH 5 - PROGRESS: at 56.36% examples, 155246 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:53:17,325 : EPOCH 5 - PROGRESS: at 56.50% examples, 155265 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:53:18,350 : EPOCH 5 - PROGRESS: at 56.61% examples, 155287 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:53:19,358 : EPOCH 5 - PROGRESS: at 56.72% examples, 155304 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:53:20,391 : EPOCH 5 - PROGRESS: at 56.83% examples, 155304 words/s, in_qsize 8, out_qsize 0
2021-08-19 13:53:21,418 : EPOCH 5 - PROGRESS: at 56.94% examples, 155324 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:53:22,429 : EPOCH 5 - PROGRESS: at 57.11% examples, 155334 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:53:23,435 : EPOCH 5 - PROGRESS: at 57.37% examples, 155355 words/s, in_qsize 8, out_qsize 0
2021-08-19 13:53:24,466 : EPOCH 5 - PROGRESS: 

2021-08-19 13:54:36,907 : EPOCH 5 - PROGRESS: at 70.08% examples, 156129 words/s, in_qsize 8, out_qsize 0
2021-08-19 13:54:37,949 : EPOCH 5 - PROGRESS: at 70.19% examples, 156115 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:54:38,964 : EPOCH 5 - PROGRESS: at 70.28% examples, 156097 words/s, in_qsize 8, out_qsize 0
2021-08-19 13:54:39,970 : EPOCH 5 - PROGRESS: at 70.38% examples, 156108 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:54:40,989 : EPOCH 5 - PROGRESS: at 70.54% examples, 156120 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:54:41,991 : EPOCH 5 - PROGRESS: at 70.62% examples, 156120 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:54:43,006 : EPOCH 5 - PROGRESS: at 70.73% examples, 156141 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:54:44,070 : EPOCH 5 - PROGRESS: at 70.83% examples, 156146 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:54:45,144 : EPOCH 5 - PROGRESS: at 70.96% examples, 156157 words/s, in_qsize 8, out_qsize 0
2021-08-19 13:54:46,386 : EPOCH 5 - PROGRESS: 

2021-08-19 13:55:58,671 : EPOCH 5 - PROGRESS: at 79.25% examples, 156568 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:55:59,711 : EPOCH 5 - PROGRESS: at 79.38% examples, 156578 words/s, in_qsize 8, out_qsize 0
2021-08-19 13:56:00,777 : EPOCH 5 - PROGRESS: at 79.48% examples, 156579 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:56:01,783 : EPOCH 5 - PROGRESS: at 79.55% examples, 156562 words/s, in_qsize 8, out_qsize 0
2021-08-19 13:56:02,860 : EPOCH 5 - PROGRESS: at 79.67% examples, 156574 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:56:03,869 : EPOCH 5 - PROGRESS: at 79.81% examples, 156600 words/s, in_qsize 8, out_qsize 0
2021-08-19 13:56:04,876 : EPOCH 5 - PROGRESS: at 79.95% examples, 156609 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:56:06,052 : EPOCH 5 - PROGRESS: at 80.09% examples, 156592 words/s, in_qsize 6, out_qsize 1
2021-08-19 13:56:07,100 : EPOCH 5 - PROGRESS: at 80.24% examples, 156609 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:56:08,108 : EPOCH 5 - PROGRESS: 

2021-08-19 13:57:20,361 : EPOCH 5 - PROGRESS: at 88.82% examples, 156860 words/s, in_qsize 8, out_qsize 0
2021-08-19 13:57:21,374 : EPOCH 5 - PROGRESS: at 88.91% examples, 156866 words/s, in_qsize 8, out_qsize 0
2021-08-19 13:57:22,395 : EPOCH 5 - PROGRESS: at 89.01% examples, 156866 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:57:23,507 : EPOCH 5 - PROGRESS: at 89.14% examples, 156872 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:57:24,543 : EPOCH 5 - PROGRESS: at 89.31% examples, 156883 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:57:25,559 : EPOCH 5 - PROGRESS: at 89.41% examples, 156888 words/s, in_qsize 8, out_qsize 0
2021-08-19 13:57:26,571 : EPOCH 5 - PROGRESS: at 89.54% examples, 156894 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:57:27,624 : EPOCH 5 - PROGRESS: at 89.65% examples, 156890 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:57:28,767 : EPOCH 5 - PROGRESS: at 89.77% examples, 156890 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:57:29,840 : EPOCH 5 - PROGRESS: 

2021-08-19 13:58:41,581 : EPOCH 5 - PROGRESS: at 97.45% examples, 157172 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:58:42,604 : EPOCH 5 - PROGRESS: at 97.54% examples, 157167 words/s, in_qsize 8, out_qsize 0
2021-08-19 13:58:43,700 : EPOCH 5 - PROGRESS: at 97.63% examples, 157164 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:58:44,751 : EPOCH 5 - PROGRESS: at 97.73% examples, 157160 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:58:45,785 : EPOCH 5 - PROGRESS: at 97.81% examples, 157155 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:58:46,816 : EPOCH 5 - PROGRESS: at 97.92% examples, 157162 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:58:47,867 : EPOCH 5 - PROGRESS: at 98.03% examples, 157171 words/s, in_qsize 7, out_qsize 0
2021-08-19 13:58:48,883 : EPOCH 5 - PROGRESS: at 98.14% examples, 157180 words/s, in_qsize 8, out_qsize 0
2021-08-19 13:58:49,969 : EPOCH 5 - PROGRESS: at 98.27% examples, 157189 words/s, in_qsize 8, out_qsize 0
2021-08-19 13:58:51,079 : EPOCH 5 - PROGRESS: 

In [9]:
def searching():
    model = word2vec.Word2Vec.load("ENtest.model")
    word = model.wv.most_similar(u'李知恩',topn=20)
    for t in word:
        print(t[0],t[1])

searching()

2021-08-19 14:03:29,108 : loading Word2Vec object from ENtest.model
2021-08-19 14:03:29,428 : loading wv recursively from ENtest.model.wv.* with mmap=None
2021-08-19 14:03:29,429 : loading vectors from ENtest.model.wv.vectors.npy with mmap=None
2021-08-19 14:03:30,424 : loading syn1neg from ENtest.model.syn1neg.npy with mmap=None
2021-08-19 14:03:31,434 : setting ignored attribute cum_table to None
2021-08-19 14:03:38,135 : Word2Vec lifecycle event {'fname': 'ENtest.model', 'datetime': '2021-08-19T14:03:38.135009', 'gensim': '4.0.1', 'python': '3.7.9 (v3.7.9:13c94747c7, Aug 15 2020, 01:31:08) \n[Clang 6.0 (clang-600.0.57)]', 'platform': 'Darwin-20.5.0-x86_64-i386-64bit', 'event': 'loaded'}


申升勳 0.8706851601600647
崔泰俊 0.8663108348846436
鄭乙英 0.8648756742477417
李章宇 0.8641693592071533
車藝蓮 0.863105058670044
南志鉉 0.8627200722694397
李孝濟 0.8622995615005493
池秀媛 0.8593701720237732
李廷鎮 0.8569570779800415
李侑菲 0.8546416759490967
朴真熙 0.8544878959655762
炫彬 0.8535040020942688
朴志胤 0.8526301383972168
任昌丁 0.8516265153884888
裴秀彬 0.8505257964134216
柳孝榮 0.8503376245498657
金浩鎮 0.8499509692192078
金智恩 0.848289430141449
孔升妍 0.8481590151786804
姜惠貞 0.8456966876983643
